# Mid Term Exam

### Import packages

In [1]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [2]:
clstr = Cluster()
session = clstr.connect()

### Use session to 'talk' to cassandra

In [3]:
session.execute("DROP KEYSPACE IF EXISTS m14")

In [29]:
session.execute("DROP KEYSPACE IF EXISTS midterm_simran")

In [30]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}") #executed the cql command at the cassandra node too and saw that m14 keyspace has been deleted

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


In [32]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS midterm_simran WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':3
    }
""")

In [25]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

midterm_simran
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


In [33]:
session.execute("""
CREATE TABLE IF NOT EXISTS midterm_simran.proposed_sys ( 
    userID INT, 
    date TEXT, 
    file_type TEXT,
    PRIMARY KEY(userID)
);
""")

What made me select userID as primary key?
-userID will be the unique identifier for each record which keeps a track of attached text file for each chat transaction.

### Load data

In [37]:
df = pd.read_csv('bd_midterm_csv.csv')
df

,userID,date,file_type
0,101,'10-04-23','text'
1,102,'09-04-23','image'


In [39]:
for index, row in df.iterrows():
    print(f"userID = {row.userID}, date={row.date}, file_Type = {row.file_type}")

userID = 101, date='10-04-23', file_Type = 'text'
userID = 102, date='09-04-23', file_Type = 'image'


In [40]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO midterm_simran.proposed_sys (userID,date,file_type)     
        VALUES ({row[0]}, {row[1]}, {row[2]});
        """
       )
    session.execute(f"""
        INSERT INTO midterm_simran.proposed_sys (userID,date,file_type)     
        VALUES ({row[0]}, {row[1]}, {row[2]});
        """
       )


        INSERT INTO midterm_simran.proposed_sys (userID,date,file_type)     
        VALUES (101, '10-04-23', 'text');
        

        INSERT INTO midterm_simran.proposed_sys (userID,date,file_type)     
        VALUES (102, '09-04-23', 'image');
        


In [41]:
rows = session.execute("select (userID,date,file_type) from midterm_simran.proposed_sys")
for row in rows:
    print(f"userID={row[0][0]}, date={row[0][1]}, file_Type={row[0][2]}")


userID=102, date=09-04-23, file_Type=image
userID=101, date=10-04-23, file_Type=text


In [42]:
rows = session.execute("select (userID,date,file_type) from midterm_simran.proposed_sys where userID  = 101")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}")

101, 10-04-23, text


Approach for the question:
>Create a text file and then read it into a tmp variable

>append that variable to the already existing dataframe formed with the csv that has earlier data

>using that new dataframe to make table on the cluster

>create function that will execute CQL command to retrieve data by executing select query

>create function that will execute CQL command to delete data by executing delete query


In [49]:
with open("abc.txt:","w") as fout:
    fout.write("Hello Prof.")
with open("abc.txt:","r") as fin:
    tmp= fin.read()
tmp

'Hello Prof.'

In [50]:
session.execute("""
CREATE TABLE IF NOT EXISTS midterm_simran.proposed_sys2 ( 
    userID INT, 
    date TEXT, 
    file_type TEXT,
    actual_type TEXT,
    PRIMARY KEY(userID)
);
""")

In [51]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO midterm_simran.proposed_sys2 (userID,date,file_type)     
        VALUES ({row[0]}, {row[1]}, {row[2]});
        """
       )
    session.execute(f"""
        INSERT INTO midterm_simran.proposed_sys2 (userID,date,file_type)     
        VALUES ({row[0]}, {row[1]}, {row[2]});
        """
       )


        INSERT INTO midterm_simran.proposed_sys2 (userID,date,file_type)     
        VALUES (101, '10-04-23', 'text');
        

        INSERT INTO midterm_simran.proposed_sys2 (userID,date,file_type)     
        VALUES (102, '09-04-23', 'image');
        


In [52]:
rows = session.execute("update midterm_simran.proposed_sys set actual_type={tmp} where userID=101")
for row in rows:
    print(f"userID={row[0][0]}, date={row[0][1]}, file_Type={row[0][2]}, actual_type={row[0][3]}")


SyntaxException: <Error from server: code=2000 [Syntax error in CQL query] message="line 1:55 no viable alternative at input '}' (update midterm_simran.proposed_sys set actual_type=[{]tmp}...)">